In [ ]:
!pip install --upgrade numpy
!pip install --upgrade pandas
#!pip install pyspark==2.4.5
!pip install -U scikit-learn
#!pip install gensim

In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from pprint import pprint
from time import time
import logging
import numpy as np
import pandas as pd
import string
import re
from datetime import datetime
from packaging import version

from ibm_botocore.client import Config
import ibm_boto3

from sklearn.model_selection import train_test_split

In [11]:
#Get our data from IBM Cloud

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_news = {
    'IAM_SERVICE_ID': 'iam-ServiceId-32e8ee67-397c-4ff1-b69b-543172331f43',
    'IBM_API_KEY_ID': 'Rx4FR4JSAueCnnIsoevsgYgOsuh8LCXtbkFpFpC0EmVU',
    #'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'ENDPOINT':'https://s3-api.us-geo.objectstorage.softlayer.net',
    'IBM_AUTH_ENDPOINT': 'https://iam.cloud.ibm.com/oidc/token',
    'BUCKET': 'advanceddatasciencecapstone-donotdelete-pr-tqabpnbxebk8rm',
    'FILE': 'dfTrueFalseNews.pkl'
}

def download_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.download_file(Bucket=credentials['BUCKET'],Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Downloaded')

def upload_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.upload_file(Filename=local_file_name, Bucket=credentials['BUCKET'],Key=key)
    except Exception as e:
        print(Exception, e)
    else:
        print(' File Uploaded')
        
dfNews = download_file_cos(credentials_news, "dfTrueFalseNews.pkl", "dfTrueFalseNews.pkl")
dfNews = download_file_cos(credentials_news, "conf.yml", "conf.yml")
dfNews = download_file_cos(credentials_news, "pipeline.py", "pipeline.py")
dfNews = download_file_cos(credentials_news, "text_preprocessing.py", "text_preprocessing.py")
dfNews = download_file_cos(credentials_news, "RNN_model.py", "RNN_model.py")


File Downloaded
File Downloaded
File Downloaded
File Downloaded
File Downloaded


In [3]:
dfNewsTemp = pd.read_pickle('dfTrueFalseNews.pkl')
#dfTrueFalseNews_tokenized = pd.read_pickle('dfTrueFalseNews_tokenized.pkl')
#dfNews['truthvalue'] = pd.Categorical(dfNews['truthvalue'])

x = dfNewsTemp['text'].values
y = dfNewsTemp['truthvalue'].values
print (dfNewsTemp.shape, dfNewsTemp.columns, '\n', dfNewsTemp.dtypes, type(x), type(y))

(1126, 3) Index(['text', 'source', 'truthvalue'], dtype='object') 
 text          object
source        object
truthvalue    object
dtype: object <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [10]:
# Get the pretrained GloVe embedding.
#!curl http://nlp.stanford.edu/data/glove.6B.zip --output glove.6B.zip -L
download_file_cos(credentials_news, "glove.6B.zip", "glove.6B.zip")
#!ls -al
!unzip glove.6B.zip glove.6B.300d.txt
#!ls -al
#upload_file_cos(credentials_news, "glove.6B.zip", "glove.6B.zip")
!rm glove.6B.zip
!ls -al

File Downloaded
Archive:  glove.6B.zip
  inflating: glove.6B.300d.txt       
total 1015844
drwxr-x--- 3 dsxuser dsxuser       4096 Sep 19 21:08 .
drwx------ 1 dsxuser dsxuser       4096 Sep 19 20:52 ..
-rw-r----- 1 dsxuser dsxuser         92 Sep 19 20:52 conf.yml
-rw-r----- 1 dsxuser dsxuser    2228457 Sep 19 20:52 dfTrueFalseNews.pkl
-rw-rw-r-- 1 dsxuser dsxuser 1037962819 Aug 27  2014 glove.6B.300d.txt
-rw-r----- 1 dsxuser dsxuser       2677 Sep 19 20:52 pipeline.py
drwxr-x--- 2 dsxuser dsxuser       4096 Sep 19 20:52 __pycache__
-rw-r----- 1 dsxuser dsxuser       1324 Sep 19 20:52 text_preprocessing.py


In [ ]:
# Package for data wrangling
import pandas as pd 
# Package for array math
import numpy as np 
# Package for system path traversal
import os
# Package for working with dates
from datetime import date
# K fold analysis package
from sklearn.model_selection import KFold
# Import the main analysis pipeline
from pipeline import Pipeline
# Tensor creation class
from text_preprocessing import TextToTensor
# Reading the configuration file
import yaml
with open("conf.yml", 'r') as file:
    conf = yaml.safe_load(file).get('pipeline')

In [12]:
# Modified, based on https://github.com/Eligijus112/twitter-genuine-tweets/blob/master/master.py
# We're not going to use any stop words. The pretrained embedding has a lot of words.
stop_words = []
#try:
#    stop_words = pd.read_csv('data/stop_words.txt', sep='\n', header=None)[0].tolist()
#except Exception as e:
    # This exception indicates that the file is missing or is in a bad format
#    print('Bad stop_words.txt file: {e}')

# Reading the data
#train = pd.read_csv('data/train.csv')[['text', 'target']]
#test = pd.read_csv('data/test.csv')

#from sklearn import model_selection
#Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(dfNewsTemp['text'],dfNewsTemp['truthvalue'],test_size=0.1)

# Creating the input for the pipeline
X_train = dfNewsTemp['text'].tolist()
Y_train = dfNewsTemp['truthvalue'].tolist()

X_test = dfNewsTemp['text'].tolist()

if conf.get('k_fold'):
    kfold = KFold(n_splits=5)
    acc = []
    f1 = []
    history =[]
    for train_index, test_index in kfold.split(X_train):
        # Fitting the model and forecasting with a subset of data
        k_results = Pipeline(
            X_train=np.array(X_train)[train_index],
            Y_train=np.array(Y_train)[train_index], 
            embed_path='glove.6B.300d.txt',
            embed_dim=300,
            X_test=np.array(X_train)[test_index],
            Y_test=np.array(Y_train)[test_index],
            max_len=conf.get('max_len'),
            epochs=conf.get('epochs'),
            batch_size=conf.get('batch_size')
        )
        # Saving the accuracy
        acc += [k_results.acc]
        f1 += [k_results.f1]
        history.append(k_results)
        print(f'The accuracy score is: {acc[-1]}') 
        print(f'The f1 score is: {f1[-1]}') 
    print(f'Total mean accuracy is: {np.mean(acc)}')
    print(f'Total mean f1 score is: {np.mean(f1)}')

# Running the pipeline with all the data
results = Pipeline(
    X_train=X_train,
    Y_train=Y_train, 
    embed_path='glove.6B.300d.txt',
    embed_dim=300,
    #stop_words=stop_words,
    X_test=X_test,
    max_len= None ,#   conf.get('max_len'),
    epochs=conf.get('epochs'),
    batch_size=conf.get('batch_size')
)

# Some sanity checks
#good = ["Fire in Vilnius! Where is the fire brigade??? #emergency"]
#bad = ["Sushi or pizza? Life is hard :(("]

TextToTensor_instance = TextToTensor(
tokenizer=results.tokenizer,
max_len=conf.get('max_len')
)

# Converting to tensors
good_nn = TextToTensor_instance.string_to_tensor(good)
bad_nn = TextToTensor_instance.string_to_tensor(bad)

# Forecasting
p_good = results.model.predict(good_nn)[0][0]
p_bad = results.model.predict(bad_nn)[0][0]

print(f'Sentence: {good_nn} Score: {p_good}')
print(f'Sentence: {bad_nn} Score: {p_bad}')

# Saving the predictions
test['prob_is_genuine'] = results.yhat
test['target'] = [1 if x > 0.5 else 0 for x in results.yhat]
 
# Saving the predictions to a csv file
if conf.get('save_results'):
    if not os.path.isdir('output'):
        os.mkdir('output')    
    test[['id', 'target']].to_csv(f'output/submission_{date.today()}.csv', index=False)

results.save("revisedDeepLearningPretrainedEmbed.model")
upload_file_cos(credentials_news, "revisedDeepLearningPretrainedEmbed.model", "revisedDeepLearningPretrainedEmbed.model")

import pickle
filename = "revisedDeepLearningPretrainedEmbed.history"
with open(filename, 'wb') as f:
    pickle.dump(history_dict, f)

FileNotFoundError: [Errno 2] No such file or directory: 'data/train.csv'

In [8]:


import text_preprocessing

Using TensorFlow backend.
/opt/conda/envs/Python36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/envs/Python36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/envs/Python36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/envs/Python36/lib/python3.6/site-packag

In [9]:
dfNewsTemp['text'].tolist()

['A Google computer victorious over the world\'s \'Go\' champion  On Saturday, a Google computer clocked its third consecutive victory over Lee Se-dol, the long-reigning global champion of the world\'s most complex board game. That win makes the machine the clear winner in a best-of-five series.  The achievements of the Google DeepMind computer, AlphaGo, are considered a significant advancement in artificial intelligence.  "To be honest, we are a bit stunned," said Google DeepMind CEO Demis Hassabis, after AlphaGo\'s third win. "We came here to challenge Lee Se-dol, as we wanted to learn from him and see what AlphaGo was capable of.',
 'White House keeps up sanctuary cities pressure with funding threat   Washington (CNN)Attorney General Jeff Sessions outlined Monday how the Trump administration will use federal funds to crack down on "sanctuary cities" and states that choose not to comply with federal immigration laws, as it has threatened to do since January.  The comments came after 